In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [16]:
def train_mnist_conv():
    mnist = keras.datasets.mnist
    (training_images, training_labels), (testing_images, testing_labels) = mnist.load_data(path="mnist.npz")
    callbacks = MyCallbacks()
    
    training_images = training_images.reshape(60000, 28, 28, 1)
    testing_images = testing_images.reshape(10000, 28, 28, 1)
    
    training_images = training_images / 255.0
    testing_images = testing_images / 255.0
    
    model = keras.models.Sequential([
        keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(28,28,1)),
        keras.layers.MaxPooling2D(2,2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dense(10, activation="softmax")
    ])
    
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
    model.evaluate(testing_images, testing_labels)

In [17]:
train_mnist_conv()

Train on 60000 samples
10000/10000 [==============================] - 1s 130us/sample - loss: 2.2979 - acc: 0.1330


In [15]:
class MyCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log={}):
        if(log.get("acc") > 0.99):
            print("\nAccuracy reached 99% so cancelling training")
            self.model.stop_training = True